In [1]:
import pandas as pd
import sys
import os

sys.path.append(f'/home/madams01/tm/ml-tox/src/')
from mltox.db.mongo import *
from mltox.db.bc import *
from mltox.db.bio import *
from mltox.db.graph_utils import *
import deepchem as dc
import numpy as np

%load_ext autoreload

2023-03-28 11:07:18.625019: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-28 11:07:18.625082: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-03-28 11:07:18.664772: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-03-28 11:07:20.079214: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-03-28 11:07:20.105391: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: ca

In [2]:
DB5 = openMongo(db='genra_dev_v5',host='pb.epa.gov',auth=True)
Assay_list = get_bio_assays(DB5.toxcast_assays)
prefixes = set([i.split('_')[0] for i in Assay_list])
prefix = 'ATG'
subset_assays = [i for i in Assay_list if i.startswith(prefix)]

if os.path.exists('x_with_dtx.pkl') and os.path.exists('y_with_dtx.pkl'):
    X = pd.read_pickle('x_with_dtx.pkl')
    Y = pd.read_pickle('y_with_dtx.pkl')
else:
    X,Y = get_task_data(subset_assays,keep_dtx=True)
    X.to_pickle('x_with_dtx.pkl')
    Y.to_pickle('y_with_dtx.pkl')

B1.1
B1.1
B1.1
B1.1
B1.1
B1.1
B1.1
B1.1
B1.1
B1.1
B1.1
B1.1
B1.1
B1.1
B1.1
B1.1
B1.1
B1.1
B1.1
B1.1
B1.1
B1.1
B1.1
B1.1
B1.1
B1.1
B1.1
B1.1
B1.1
B1.1
B1.1
B1.1
B1.1
B1.1
B1.1
B1.1
B1.1
B1.1
B1.1
B1.1
B1.1
B1.1
B1.1
B1.1
B1.1
B1.1
B1.1
B1.1
B1.1
B1.1
B1.1
B1.1
B1.1
B1.1
B1.1
B1.1
B1.1
B1.1
B1.1
B1.1
B1.1
B1.1
B1.1
B1.1
B1.1
B1.1
B1.1
B1.1
B1.1
B1.1
B1.1
B1.1
B1.1
B1.1
B1.1
B1.1
B1.1
B1.1
B1.1
B1.1
B1.1
B1.1
B1.1
B1.1
B1.1
B1.1
B1.1
B1.1
B1.1
B1.1
B1.1
B1.1
B1.1
B1.1
B1.1
B1.1
B1.1
B1.1
B1.1


In [3]:
featurizer = dc.feat.MolGraphConvFeaturizer(use_edges=True)
X1,Y1, smiles = task_to_graph(X,Y,featurizer,dtx=True)
Y_new = replace_missing_data(Y1,"fill")
dataset = dc.data.NumpyDataset(X=X1,y=Y_new)

Failed to featurize datapoint 243, [Br-].[Br-].[Ca+2]. Appending empty array
Exception message: zero-size array to reduction operation maximum which has no identity
Failed to featurize datapoint 271, [Cl-].[Cl-].[Cl-].[Yb+3]. Appending empty array
Exception message: zero-size array to reduction operation maximum which has no identity
Failed to featurize datapoint 912, [Cl-].[Cl-].[Zn+2]. Appending empty array
Exception message: zero-size array to reduction operation maximum which has no identity
Failed to featurize datapoint 931, [I-].[Na+]. Appending empty array
Exception message: zero-size array to reduction operation maximum which has no identity
Failed to featurize datapoint 1504, [Cl-].[Cl-].[Pd+2]. Appending empty array
Exception message: zero-size array to reduction operation maximum which has no identity
Failed to featurize datapoint 1768, [Cl-].[Cl-].[Hg+2]. Appending empty array
Exception message: zero-size array to reduction operation maximum which has no identity
Failed to 

ValueError: too many values to unpack (expected 3)

In [ ]:
splitter = dc.splits.RandomStratifiedSplitter()
train_dataset, _ ,test_dataset = splitter.train_valid_test_split(
                                                                            dataset,
                                                                            frac_train=0.8,
                                                                            frac_test=0.2,
                                                                            frac_valid=0.0,
                                                                            seed=0,               
                                                                            test_dir=f'{os.getcwd()}/tmp_test_data/',
                                                                            train_dir=f'{os.getcwd()}/tmp_train_data/',
                                                                            valid_dir=f'{os.getcwd()}/tmp_valid_data/'
                                                                            )
                                                                            

In [5]:
tsca = pd.read_pickle('tsca_probs.pkl')
tsca_fps  = get_chm_FP(
                tsca.DTXSID,
                DB5.chms_fp,
                fp='mrgn',
                fill=np.uint(0)
                    )

In [6]:
tsca = tsca[['DTXSID']].rename(columns={'DTXSID':'dsstox_sid'})

In [11]:
tsca_fps = tsca.merge(tsca_fps,on='dsstox_sid',how='left')

In [12]:
train_fps  = get_chm_FP(
                train_dataset.X,
                DB5.chms_fp,
                fp='mrgn',
                fill=np.uint(0)
                    )

In [13]:
# Make fingerprints same length
tmp = pd.concat([train_fps.head(1),tsca_fps]).fillna(0)
tsca_fps = tmp.iloc[1:,:]

In [10]:
train_fps['label'] = train_dataset.y[:,53]

pos_train_fps = train_fps[train_fps['label']==1].iloc[:,:-1]

In [22]:
pos_train_fps

,dsstox_sid,mrgn_0,mrgn_1,mrgn_10,mrgn_100,mrgn_1000,mrgn_1001,mrgn_1002,mrgn_1003,mrgn_1004,...,mrgn_990,mrgn_991,mrgn_992,mrgn_993,mrgn_994,mrgn_995,mrgn_996,mrgn_997,mrgn_998,mrgn_999
3,DTXSID0020284,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,DTXSID0020365,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
20,DTXSID0020652,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26,DTXSID0021094,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
29,DTXSID0021125,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2752,DTXSID9047542,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2753,DTXSID9047546,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2757,DTXSID9047598,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2758,DTXSID9047623,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [44]:
from sklearn.neighbors import NearestNeighbors

neigh = NearestNeighbors(n_neighbors=1,metric='jaccard')
neigh.fit(pos_train_fps.iloc[:,1:])


NearestNeighbors(metric='jaccard', n_neighbors=1)

In [45]:
distances,indices = neigh.kneighbors(tsca_fps.iloc[:,1:])

/home/madams01/anaconda3/envs/tflow_dc_edit/lib/python3.9/site-packages/sklearn/metrics/pairwise.py:1776: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


In [46]:
tsca = pd.read_pickle('tsca_probs.pkl')

In [47]:
def assign_label(x):
    if x > 0.5:
        return 1
    elif x < 0.5:
        return 0

def prob_predict(x):
    if x > 0.5:
        return x
    else:
        return 1-x


labels_df = Y[['dsstox_sid','ATG_PPARa_TRANS']]
labels_df = labels_df.set_index('dsstox_sid')

tsca['predicted label'] = tsca['probability_active'].apply(assign_label)
tsca['probability label'] = tsca['probability_active'].apply(prob_predict)


tsca['jaccard similarity'] = 1-distances[:,0]


tsca['train index'] = indices[:,0]

df = pos_train_fps

tsca['train index'] = tsca['train index'].apply(lambda x: df.iloc[x].name)
tsca['train dtxsid'] = tsca['train index'].apply(lambda x: df[['dsstox_sid']].iloc[x].values[0])
tsca['train label'] = tsca['train dtxsid'].apply(lambda x: labels_df.loc[x].values[0])

In [48]:
tsca

,DTXSID,probability_active,predicted label,probability label,jaccard similarity,train index,train dtxsid,train label
0,DTXSID7059777,0.063887,0,0.936113,0.185185,274,DTXSID1020308,1
1,DTXSID8061719,0.108969,0,0.891031,0.181818,996,DTXSID3037757,1
2,DTXSID1062752,0.068079,0,0.931921,0.115385,1056,DTXSID3044889,1
3,DTXSID1062178,0.045932,0,0.954068,0.440000,996,DTXSID3037757,1
4,DTXSID8059390,0.034706,0,0.965294,0.428571,2566,DTXSID9021922,1
...,...,...,...,...,...,...,...,...
337,DTXSID8047503,0.199892,0,0.800108,0.560000,1577,DTXSID5041275,1
338,DTXSID2070219,0.550984,1,0.550984,0.400000,1084,DTXSID3047558,1
339,DTXSID5044942,0.503278,1,0.503278,0.222222,2054,DTXSID7024950,1
340,DTXSID2052732,0.660907,1,0.660907,0.111111,1140,DTXSID4021557,1


In [25]:
tsca.to_pickle('tsca_data.pkl')

In [26]:
tsca[tsca['DTXSID']=='DTXSID8060905']

,DTXSID,probability_active,predicted label,probability label,jaccard similarity,train index,train dtxsid,train label
321,DTXSID8060905,0.770659,1,0.770659,0.137931,773,DTXSID2044848,1


In [19]:
example1 = tsca_fps[tsca_fps['dsstox_sid']=='DTXSID8060905']

In [33]:
sum(tsca_fps.columns == train_fps.columns)

2049

In [20]:
neigh.kneighbors(example1.iloc[:,1:])

/home/madams01/anaconda3/envs/tflow_dc_edit/lib/python3.9/site-packages/sklearn/metrics/pairwise.py:1776: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


(array([[0.86206897, 0.86419753, 0.86516854, 0.86516854, 0.86585366]]),
 array([[ 773, 2521,  149, 1995,  685]]))